In [1]:
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from segmentation.scr.utils.utils import set_seed
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
import segmentation_models_pytorch as smp
from segmentation.scr.utils import losses, transforms
from segmentation.models.unet import unet

from segmentation.scr.utils import losses, transforms
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.utils.metrics import dice_coef
from segmentation.scr.utils.utils import save_model

c:\Users\123\Desktop\human vasculature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def norm_with_clip(x:torch.Tensor,smooth=1e-5):
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    x=(x-mean)/(std+smooth)
    x[x>5]=(x[x>5]-5)*1e-3 +5
    x[x<-3]=(x[x<-3]+3)*1e-3-3
    return x

In [25]:
norm_by_percentile(im1.to(torch.float).numpy())

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [52]:
def norm_by_percentile_(volume, low=10, high=99.8, alpha=0.01, smooth=1e-7):
    dim=list(range(1,volume.ndim))
    xmin = torch.quantile(volume,low / 100 , dim=dim)
    xmax = torch.quantile(volume,high, dim=dim)
    print(xmax)
    x = (volume-xmin)/(xmax-xmin + smooth)
    x[x>1]=(x[x>1]-1)*alpha +1
    x[x<0]=(x[x<0])*alpha
    #x = np.clip(x,0,1)
    return x

In [67]:
z = im1.unsqueeze(0).unsqueeze(0).repeat(3,3,1,1).to(torch.float)
#x.reshape(-1, *x.shape[2:])

In [71]:
torch.quantile(z.reshape(-1,1), 0.1)

RuntimeError: quantile() input tensor is too large

In [72]:
z.reshape(-1,1).shape

torch.Size([23184540, 1])

In [53]:
norm_by_percentile_(

TypeError: quantile() received an invalid combination of arguments - got (Tensor, float, dim=list), but expected one of:
 * (Tensor input, Tensor q, int dim, bool keepdim, *, str interpolation, Tensor out)
 * (Tensor input, float q, int dim, bool keepdim, *, str interpolation, Tensor out)


In [66]:
dim=list(range(1,im1.unsqueeze(0).unsqueeze(0).repeat(3,3,1,1).ndim))
torch.quantile(im1.unsqueeze(0).unsqueeze(0).repeat(3,3,1,1).to(torch.float), 0.1).shape

RuntimeError: quantile() input tensor is too large

In [63]:
volume = im1.unsqueeze(0).unsqueeze(0).repeat(3,3,1,1).to(torch.float).numpy()

In [62]:
np.percentile(volume,0.1)

74.0

In [51]:
im1

tensor([[75, 75, 75,  ..., 75, 75, 75],
        [75, 75, 75,  ..., 75, 75, 75],
        [75, 75, 75,  ..., 75, 75, 75],
        ...,
        [75, 75, 75,  ..., 75, 75, 75],
        [75, 75, 75,  ..., 75, 75, 75],
        [75, 75, 75,  ..., 75, 75, 75]], dtype=torch.uint8)

In [28]:
torch.quantile(im1.to(torch.float),0.1)

tensor(75.)

In [3]:
class CFG:
    # ============== pred target =============
    target_size = 1

    # ============== model CFG =============
    model_name = 'Unet'
    backbone = 'mobilenet_v2'

    device = "cuda" if torch.cuda.is_available() else "cpu"
    in_chans = 5 # 65
    # ============== training CFG =============
    image_size = 512
    input_size=512

    valid_batch_size = 4
    lr = 6e-4
    chopping_percentile=1e-3
    # ============== fold =============
    valid_id = 1

    # ============== augmentation =============
   
    valid_aug_list = [
        ToTensorV2(transpose_mask=True),
    ]
    valid_aug = A.Compose(valid_aug_list, p=1.0)

In [4]:
path_img_dir="data\\train\\kidney_3_sparse\\images"
path_lb_dir="data\\train\\kidney_3_dense\\labels"
path_img_dir = Path(path_img_dir)
path_lb_dir = Path(path_lb_dir)

path_img_dir = sorted(list(path_img_dir.rglob("*.tif")))
path_lb_dir = sorted(list(path_lb_dir.rglob("*.tif")))

images_labels = defaultdict(list)
for img in path_img_dir:
    images_labels[img.name].append(img)
for lb in path_lb_dir:
    images_labels[lb.name].append(lb)
new_dict = dict(filter(lambda item: len(item[1]) > 1, images_labels.items()))
print(f"Общее число изображений с масками сегментации : {len(new_dict)}")
img_path = list(map(lambda key : str(new_dict[key][0]) , new_dict))
lb_path = list(map(lambda key : str(new_dict[key][1]), new_dict))

Общее число изображений с масками сегментации : 501


In [6]:
class Data_loader(Dataset):
    def __init__(self,paths,is_label):
        self.paths=paths
        self.paths.sort()
        self.is_label=is_label
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        img=torch.from_numpy(img)
        if self.is_label:
            img=(img!=0).to(torch.uint8)*255
        else:
            img=img.to(torch.uint8)
        return img

In [7]:
def resize_to_size( img, image_size = CFG.image_size):
        '''
        resizes the image if its side is smaller than image_size
        padding is the average between the maximum and minimum pixels on the border
        '''
        #print(img.shape)
        if image_size > img.shape[0]:
            #print(1)
            start = ( image_size - img.shape[0])//2
            val1 = (min(img[0,:]).item() + max(img[0,:]).item()) // 2
            val2 = (min(img[img.shape[0] - 1,:]).item() + max(img[img.shape[0] - 1,:]).item()) // 2
            print(val1, val2)
            top= torch.full((start,img.shape[1]),(val1 + val2)//2).to(torch.uint8)
            botton = torch.full((start,img.shape[1]),(val1 + val2)//2).to(torch.uint8)
            #print(border.shape)
            img=torch.cat((top, img, botton), axis = 0).to(torch.uint8)
        if image_size > img.shape[1]:
            #print(2)
            start = ( image_size - img.shape[1])//2
            val1 = (min(img[:, 0]).item() + max(img[:, 0]).item()) // 2
            val2 = (min(img[:, img.shape[1] - 1]).item() + max(img[:, img.shape[1] - 1]).item()) // 2
            left= torch.full((img.shape[0], start),(val1 + val2)//2).to(torch.uint8)
            right = torch.full((img.shape[0], start),(val1 + val2)//2).to(torch.uint8)
            img = torch.cat((left, img, right), axis = 1).to(torch.uint8)
        
        return img

In [9]:
def load_data(paths,is_label=False):
    data_loader=Data_loader(paths,is_label)
    data_loader=DataLoader(data_loader, batch_size=16)
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    x=torch.cat(data,dim=0)
    del data
    if not is_label:
      #  ########################################################################
      x = filter_noise(x)
      x=(min_max_normalization(x.to(torch.float16)[None])[0]*255).to(torch.uint8)
    return x

In [ ]:
#v#al_x=load_data(img_path ,is_label=False)
#print(val_x.shape)
#val_y=load_data(lb_path ,is_label=True)
#print(val_y.shape)

In [10]:
class Kaggld_Dataset(Dataset):
    def __init__(self,x:list,y:list,arg=False):
        super(Dataset,self).__init__()
        self.x=x#list[(C,H,W),...]
        self.y=y#list[(C,H,W),...]
        self.image_size=CFG.image_size
        self.in_chans=CFG.in_chans
        self.arg=arg
        if arg:
            self.transform=0
        else: 
            self.transform=CFG.valid_aug
            
    def __len__(self) -> int:
        return sum([y.shape[0]-self.in_chans for y in self.y])
    
    def __getitem__(self,index):
        i=0
        for x in self.x:
            if index>x.shape[0]-self.in_chans:
                index-=x.shape[0]-self.in_chans
                i+=1
            else:
                break
        x=self.x[i]
        y=self.y[i]
        
        x_index=np.random.randint(0,x.shape[1]-self.image_size)
        y_index=np.random.randint(0,x.shape[2]-self.image_size)

        x=x[index:index+self.in_chans,x_index:x_index+self.image_size,y_index:y_index+self.image_size]
        y=y[index+self.in_chans//2,x_index:x_index+self.image_size,y_index:y_index+self.image_size]

        data = self.transform(image=x.numpy().transpose(1,2,0), mask=y.numpy())
        x = data['image']
        y = data['mask']>=127
       
        if self.arg:
            if np.random.randint(2):
                x=x.flip(dims=(0,))
                   
                        
        return x,y#(uint8,uint8)

In [11]:
class Data_loader_inference(Dataset):
    def __init__(self,img_path,lb_path):
        self.img_paths=img_path
        self.lb_path = lb_path
        
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.img_paths[index],cv2.IMREAD_GRAYSCALE)
        img=torch.from_numpy(img).to(torch.uint8)
        label = cv2.imread(self.lb_path[index],cv2.IMREAD_GRAYSCALE)
            
        label=torch.from_numpy(label!=0).to(torch.uint8)*255
        return img, label

In [12]:
loader_inf = Data_loader_inference(img_path, lb_path)

In [13]:
im1 = loader_inf[0][0]

In [ ]:
plt.imshow(resize_to_size(loader_inf[0][0]))

In [ ]:
mask = resize_to_size(loader_inf[0][1])

In [ ]:
im1.shape

In [ ]:
def create_tillings( images, overlap_pct = 50):
    if len(images.shape) == 2:
        #print('ss')
        images = images.unsqueeze(0).repeat(CFG.in_chans,1,1)
    min_overlap = float(overlap_pct) * 0.01
    max_stride = CFG.image_size * (1.0 - min_overlap)

    print(images.shape)
    height, width = images.shape[-2], images.shape[-1]
    num_patches = np.ceil(np.array([height, width]) / max_stride).astype(
                np.int32
            )

    starts = [
                np.int32(np.linspace(0, height - CFG.input_size, num_patches[0])),
                np.int32(np.linspace(0, width - CFG.input_size, num_patches[1])),
            ]
    stops = [starts[0] + CFG.input_size, starts[1] + CFG.input_size]
        
    indexs=[]
    tills=[]
    for y1, y2 in zip(starts[0], stops[0]):
        for x1, x2 in zip(starts[1], stops[1]):
            tills.append(images[...,y1:y2, x1:x2])
            indexs.append((y1,y2,x1,x2))
                
  
    return tills, indexs
                
im1 = loader_inf[0][0]
#im1 = filter_noise(im1)
im1 = (min_max_normalization(im1.to(torch.float16)[None])[0]*255).to(torch.uint8)

tills, indexs = create_tilling(images=im1.repeat(CFG.in_chans,1,1))

In [ ]:
tills = torch.stack(tills)

In [ ]:
tills =tills.to(torch.float32)

In [ ]:
tills.shape

In [ ]:
`tills = norm_with_clip(tills)`

In [ ]:
model.eval()
with torch.no_grad():
    ans= model(tills.cuda().to(torch.float32))

In [ ]:
ans = ans.cpu()

In [ ]:
ans = ans.sigmoid()

In [ ]:
mask_pred = torch.zeros_like(im1,dtype=torch.float32)
mask_count = torch.zeros_like(im1,dtype=torch.float32)

In [ ]:
mask_pred.shape

In [ ]:
for i,(y1,y2,x1,x2) in enumerate(indexs):
    mask_pred[y1:y2, x1:x2] += ans[i]
    mask_count[y1:y2, x1:x2] += 1
 
mask_pred /= mask_count
 

In [ ]:
mask_pred.max()

In [ ]:
mask_pred

In [ ]:
mask.max()

In [ ]:
plt.imshow((mask_pred * 255).to(torch.uint8))

In [ ]:
def dice_coef(
    y_pred: torch.Tensor,
    y_true: torch.Tensor,
    thr=0.5,
    dim=(-1, -2),
    smooth=0.0000001
    # epsilon=1e-7,
):
    """Calculate Dice coef

    Args:
        y_pred (torch.Tensor): predicitions from logits
        y_true (torch.Tensor): labels
        thr (float, optional): cutoff threshold. Defaults to 0.5.
        dim (tuple, optional): dimensions. Defaults to (-1, -2).
        smooth (int, optional): smooth. Defaults to 0.

    Returns:
        _type_: _description_
    """
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred > thr).to(torch.float32)
    inter = (y_true * y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2 * inter + smooth) / (den + smooth)
            ).mean()  # .clamp_min(epsilon)
    return dice


In [ ]:
dice_coef(mask_pred, y, thr=0.5)

In [ ]:
plt.imshow(mask)

In [ ]:
(mask_pred*255 /3).to(torch.uint8).max()

In [ ]:
def in

In [ ]:
model=build_model(weight="imagenet")


In [ ]:
path_to_model = 'C:\\Users\\123\\Desktop\\resnet baseline\\state_mobnet_base_my_pc.pth'

In [ ]:
if not torch.cuda.is_available() or CFG.device == "cpu":
    ckpt = torch.load(path_to_model, map_location=torch.device("cpu"))
elif CFG.device == "cuda" and torch.cuda.is_available():
    ckpt = torch.load(path_to_model)

ckpt = ckpt["model"]
model.load_state_dict(ckpt)

In [ ]:
model.eval()
timer = tqdm(range(len(val_loader)))
val_losss = 0
val_scores = 0
for i, (x, y) in enumerate(val_loader):
    x = x.cuda().to(torch.float32)
    y = y.cuda().to(torch.float32)
    x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)

    with torch.no_grad():
        pred = model(x)
        loss = loss_fn(pred, y)
    score = dice_coef(pred.detach(), y)
    val_losss = (val_losss*i+loss.item())/(i+1)
    val_scores = (val_scores*i+score)/(i+1)
    timer.set_description(
            f"val-->loss:{val_losss:.4f},score:{val_scores:.4f}")
    timer.update()
timer.close()

In [ ]:
def evaluate_dataset_1(model, val_loader):
    model.eval()
    timer = tqdm(range(len(val_loader)))
    val_scores = 0
    scores = []
    for i, (x, y) in enumerate(val_loader):
        x = x.cuda().to(torch.float32)
        y = y.cuda().to(torch.float32)
        x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)

        with torch.no_grad():
            pred = model(x)

        score = dice_coef(pred.detach(), y)
        scores.append(score.detach().cpu().item())
        val_scores = (val_scores * i + score) / (i + 1)
        timer.set_description(f"eval--> score:{val_scores:.4f}")
        timer.update()
    timer.close()
    return scores, val_scores

In [ ]:
0.8321